# LLM Debugging with W&B Trace and LangChain

<a target="_blank" href="https://colab.research.google.com/github/wandb/weave/blob/master/examples/prompts/trace_debugging/trace_quickstart_langchain.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Use W&B Traces with LangChain to
* trace and visualize all the stages, intermediate processing calls, and other details of your LLM calls
* debug and diagnose errors in specific prompts or chain configurations
* get quick insights and more complete understanding of effective prompt langauge and patterns for your specific use case 

<img src="https://raw.githubusercontent.com/wandb/weave/master/docs/assets/traces_debug_board.png">

# Step 0: Setup

Install dependencies, authenticate with OpenAI, login to W&B so you can save and share your work.

In [ ]:
# if not already installed
!pip install -qqq wandb weave

In [1]:
import os
from getpass import getpass

if os.getenv("OPENAI_API_KEY") is None:
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

Paste your OpenAI key from: https://platform.openai.com/account/api-keys
 ········


OpenAI API key configured


In [2]:
import wandb
wandb.login()

wandb: Currently logged in as: stacey. Use `wandb login --relogin` to force relogin


True

In [3]:
import weave
from weave.monitoring.langchain import WeaveTracer

AttributeError: module 'langchain.callbacks.tracers.wandb' has no attribute '_serialize_io'

# Step 1: Configure data streaming

Configure how W&B stores all the work related to the current project.
* required: set WB_ENTITY to your wandb username or team name
* optional: rename the WB_PROJECT, the top-level directory for this work
* optional: rename the STREAM_NAME, the record table which stores the logs of LangChain calls

In [ ]:
WB_ENTITY = "stacey" # replace with your W&B entity
WB_PROJECT = "lc_trace_quickstart"
WB_STREAM = "prompts"

tracer = WeaveTracer(f"{WB_ENTITY}/{WB_PROJECT}/{WB_STREAM}")

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
tools = load_tools(["llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)

In [ ]:
questions = [
    'What is the sum of 5 and 7?',
    'What is the product of 4 and 6?',
    'What is the square root of 64?',
    'What is the area of a circle with a radius of 5?',
    'What is the slope of the line y = 3x + 2?',
    'What is the value of x in the equation 3x + 4 = 10?',
    'What is the value of y in the equation y = 2x + 1 when x = 3?',
    'What is the equation of the line that passes through the points (2, 4) and (3, 6)?',
    'What is the volume of a cube with a side length of 5?',
    'What is the value of x in the equation x2 + 3x - 4 = 0?'
]

In [ ]:
for question in questions:
    try:
        print(agent.run(question, callbacks=[tracer]))
    except Exception as e:
        print("Caught Exception:", e)